<a href="https://colab.research.google.com/github/jiw3026/DL_NLP_CODE/blob/main/04_Seq2Seq_Attention(%E1%84%8E%E1%85%A2%E1%86%BA%E1%84%87%E1%85%A9%E1%86%BA)_CLASS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 32.4 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
from konlpy.tag import Okt

import numpy as np
import warnings
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# WARNING 무시
warnings.filterwarnings('ignore')
corpus = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv')

print(corpus.info())
corpus

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB
None


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [ ]:
# config - 하이퍼 파라미터 정의
IS_TRAIN_MODE = True

NUM_SAMPLES = 2000 # Google Colab 일 경우 3,000개에서는 메모리 오버되는 현상 발생
MAX_LENGTH = 30 # padding length 

BUFFER_SIZE = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = MAX_LENGTH
UNITS = 128
SAMPLE_SIZE = 3
NUM_EPOCHS = 300

# tokenizer 
START_TOKEN = '<START>' 
END_TOKEN = '<END>'

CKPT_PATH = 'model/seq2seq-attention-checkpoint.ckpt'



In [ ]:
# Preprocessing

# dataset 개수 제한 
def prepare_corpus(corpus, NUM_SAMPLES):
    # 랜덤 샘플링 - random_state를 고정하지 않으면, inference 과정에서 에러가 생길 수 있음
    corpus = corpus.sample(n = NUM_SAMPLES, replace = False, random_state=42)
    
    texts = corpus['Q'].values
    pairs = corpus['A'].values
    return texts, pairs

# 데이터 전처리 함수 (정규식(regex expression)활용)
def clean_sentence(sentence):
    # 한글, 숫자를 제외한 모든 문자는 제거합니다.
    sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    return sentence

def process_morph(sentence, morph_tokenizer):
    return ' '.join(morph_tokenizer.morphs(sentence))

# 문장 전처리
def clean_and_morph(sentence, morph_tokenizer, is_question=True):
    # 한글 문장 전처리
    sentence = clean_sentence(sentence)
    # 형태소 변환
    sentence = process_morph(sentence, morph_tokenizer)
    # Question 인 경우, Answer인 경우를 분기하여 처리합니다.
    if is_question:
        return sentence
    else:
        # START 토큰은 decoder input에 END 토큰은 decoder output에 추가합니다.
        return ('<START> ' + sentence, sentence + ' <END>')
    
def preprocess(texts, pairs, morph_tokenizer):
    questions = []
    answer_in = []
    answer_out = []

    # 질의에 대한 전처리
    for text in texts:
        # 전처리와 morph 수행
        question = clean_and_morph(text, morph_tokenizer, is_question=True)
        questions.append(question)

    # 답변에 대한 전처리
    for pair in pairs:
        # 전처리와 morph 수행
        in_, out_ = clean_and_morph(pair, morph_tokenizer, is_question=False)
        answer_in.append(in_)
        answer_out.append(out_)
    
    return questions, answer_in, answer_out


def label_and_pad(questions, answer_in, answer_out, MAX_LENGTH):
    tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
    tokenizer.fit_on_texts(questions)
    tokenizer.fit_on_texts(answer_in)
    tokenizer.fit_on_texts(answer_out)
    
    # 치환: 텍스트를 시퀀스로 인코딩 (texts_to_sequences)
    question_sequence = tokenizer.texts_to_sequences(questions)
    answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
    answer_out_sequence = tokenizer.texts_to_sequences(answer_out)
    
    # 문장의 길이 맞추기 (pad_sequences)
    question_padded = pad_sequences(question_sequence, 
                                    maxlen=MAX_LENGTH, 
                                    truncating='post', 
                                    padding='post')
    answer_in_padded = pad_sequences(answer_in_sequence, 
                                    maxlen=MAX_LENGTH, 
                                    truncating='post', 
                                    padding='post')
    answer_out_padded = pad_sequences(answer_out_sequence, 
                                    maxlen=MAX_LENGTH, 
                                    truncating='post', 
                                    padding='post')
    
    return tokenizer, (question_padded, answer_in_padded, answer_out_padded)

def convert_to_one_hot(padded, VOCAB_SIZE):
    # 원핫인코딩 초기화
    one_hot_vector = np.zeros((len(padded), 
                               MAX_LENGTH, 
                               VOCAB_SIZE))

    # 디코더 목표를 원핫인코딩으로 변환
    # 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
    for i, sequence in enumerate(padded):
        for j, index in enumerate(sequence):
            one_hot_vector[i, j, index] = 1

    return one_hot_vector

In [ ]:
# After Training

# 변환된 index를 다시 단어로 변환
def convert_index_to_text(indexs, tokenizer, END_TOKEN_IDX): 
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_TOKEN_IDX:
            # 끝 단어이므로 예측 중비
            break;
        # 사전에 존재하는 단어의 경우 단어 추가
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
        # 사전에 없는 인덱스면 빈 문자열 추가
            sentence += ''
            
        # 빈칸 추가
        sentence += ' '
    return sentence

def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)
    return results

# 자연어 (질문 입력) 대한 전처리 함수
def make_question(sentence, tokenizer, morph_tokenizer):
    sentence = clean_and_morph(sentence, morph_tokenizer)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
    return question_padded

# 챗봇
def run_chatbot(question, model):
    question_inputs = make_question(question,tokenizer=tokenizer, morph_tokenizer = okt)
    results = make_prediction(model, question_inputs)
    results = convert_index_to_text(results, tokenizer, END_TOKEN_IDX)
    return results



In [ ]:
texts, pairs = prepare_corpus(corpus, NUM_SAMPLES)

okt = Okt()
questions, answer_in, answer_out = preprocess(texts, pairs, okt)

tokenizer, (question_padded, answer_in_padded, answer_out_padded) = label_and_pad(questions, answer_in, answer_out, MAX_LENGTH)

In [ ]:
# tf.tokenizer Config
VOCAB_SIZE = len(tokenizer.word_index)+1
START_TOKEN_IDX = tokenizer.word_index[START_TOKEN]
END_TOKEN_IDX = tokenizer.word_index[END_TOKEN]

print('vocab size :', VOCAB_SIZE)

# label -> onehot
answer_in_one_hot = convert_to_one_hot(answer_in_padded, VOCAB_SIZE)
answer_out_one_hot = convert_to_one_hot(answer_out_padded, VOCAB_SIZE)

vocab size : 4874


In [ ]:
# question와 answer 데이터 확인
print('\nprepare corpus:\n', list(zip(texts, pairs))[1995:2000])

# 전처리 함수 테스트
print('\nclean_sentence :', clean_sentence('안녕하세요~:)'), clean_sentence('텐서플로^@^%#@!'))

# Okt형태소
print('\nQ:\t',questions[:2])
print('A in:\t',answer_in[:2])
print('A out:\t',answer_out[:2])


# 단어사전 확인
print()
for word, idx in tokenizer.word_index.items():
    print(f'{word}\t -> \t{idx}')
    if idx > 5:
        break

# 토큰 갯수 확인
print("\nnumber of tokens :", len(tokenizer.word_index))

# padding 확인
print('\npadding : ', question_padded.shape, answer_in_padded.shape, answer_out_padded.shape)


# answer -> onehot
print('\nonehot 변환 :\t', answer_in_one_hot.shape, answer_out_one_hot.shape)


prepare corpus:
 [('남자친구가 돈이 너무 없어.', '정말 현실적인 문제네요.'), ('날 좋아하는 사람을 어떻게 하면 저를 좋아하게 만들까요?', '그 사람이 좋아하는 것들을 알아보세요.'), ('간만에 꿈에 나와준 그녀 덕에 추억들이 막 떠오르네', '좋은 추억만 간직하세요.'), ('짝녀가 차갑게 굴면 어떻게 해요?', '짝사랑하는 마음을 접는게 편할 것 같아요.'), ('마음을 정리하는 중인데 카톡 왔어', '정리하는데 도움이 될 수도 있겠네요.')]

clean_sentence : 안녕하세요 텐서플로

Q:	 ['죽을거 같네', '내일 시험 이야']
A in:	 ['<START> 나쁜 생각 하지 마세요', '<START> 컨디션 조절 하세요']
A out:	 ['나쁜 생각 하지 마세요 <END>', '컨디션 조절 하세요 <END>']

<OOV>	 -> 	1
<START>	 -> 	2
<END>	 -> 	3
이	 -> 	4
을	 -> 	5
거	 -> 	6

number of tokens : 4873

padding :  (2000, 30) (2000, 30) (2000, 30)

onehot 변환 :	 (2000, 30, 4874) (2000, 30, 4874)


In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, 
                                   embedding_dim, 
                                   input_length=time_steps, 
                                   name='Embedding')
        self.dropout = Dropout(0.2, name='Dropout')
        # (attention) return_sequences=True 추가
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True, 
                         name='LSTM')
        
    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        # (attention) x return 추가
        return x, [hidden_state, cell_state]

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, 
                                   embedding_dim, 
                                   input_length=time_steps, 
                                   name='Embedding')
        self.dropout = Dropout(0.2, name='Dropout')
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True, 
                         name='LSTM'
                        )
        self.attention = Attention(name='Attention')
        self.dense = Dense(vocab_size, 
                           activation='softmax', 
                           name='Dense')
    
    def call(self, inputs, initial_state):
        # (attention) encoder_inputs 추가
        encoder_inputs, decoder_inputs = inputs
        x = self.embedding(decoder_inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
        
        # (attention) key_value, attention_matrix 추가
        # 이전 hidden_state의 값을 concat으로 만들어 vector를 생성합니다.        
        key_value = tf.concat([initial_state[0][:, tf.newaxis, :], 
                               x[:, :-1, :]], axis=1)        
        # 이전 hidden_state의 값을 concat으로 만든 vector와 encoder에서 나온 
        # 출력 값들로 attention을 구합니다.
        attention_matrix = self.attention([key_value, encoder_inputs])
        # 위에서 구한 attention_matrix와 decoder의 출력 값을 concat 합니다.
        x = tf.concat([x, attention_matrix], axis=-1)
        
        x = self.dense(x)
        return x, hidden_state, cell_state

In [ ]:
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(Seq2Seq, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps
        
        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)
        
        
    def call(self, inputs, training=True):
        if training:
            encoder_inputs, decoder_inputs = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(encoder_inputs)
            # (attention) decoder 입력 값 수정
            decoder_outputs, _, _ = self.decoder((encoder_outputs, decoder_inputs), 
                                                 initial_state=context_vector)
            return decoder_outputs
        else:
            x = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(x)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)
            
            for i in tf.range(self.time_steps):
                decoder_output, decoder_hidden, decoder_cell = self.decoder((encoder_outputs, target_seq), 
                                                                            initial_state=context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)
                
                if decoder_output == self.end_token:
                    break
                    
                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]
                
            return tf.reshape(results.stack(), shape=(1, self.time_steps))

In [ ]:
# 체크포인트 생성
checkpoint = ModelCheckpoint(filepath=CKPT_PATH, 
                             save_weights_only=True,
                             save_best_only=True, 
                             monitor='loss', 
                             verbose=1
                            )


In [ ]:
class PrintingPredictionCallback(tf.keras.callbacks.Callback):
    def __init__(self, printing_epoch = 10):
        self.printing_epoch = printing_epoch
        
    def on_epoch_end(self, epoch,  logs=None):
        if epoch % self.printing_epoch == 0 : 
                # 랜덤한 샘플 번호 추출
                samples = np.random.randint(NUM_SAMPLES, size=SAMPLE_SIZE)

                # 예측 성능 테스트
                for idx in samples:
                    question_inputs = question_padded[idx]
                    # 문장 예측
                    results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))
                    
                    # 변환된 인덱스를 문장으로 변환
                    results = convert_index_to_text(results, tokenizer, END_TOKEN_IDX)
                    
                    print(f'Q: {questions[idx]}')
                    print(f'A: {results}\n')
                    
printing_cb = PrintingPredictionCallback(30)

In [ ]:
# seq2seq
seq2seq = Seq2Seq(UNITS, 
                  VOCAB_SIZE, 
                  EMBEDDING_DIM, 
                  TIME_STEPS, 
                  START_TOKEN_IDX, 
                  END_TOKEN_IDX)

seq2seq.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['acc'])

In [ ]:
if IS_TRAIN_MODE : 
    seq2seq.fit([question_padded, answer_in_padded],
                answer_out_one_hot,
                epochs=NUM_EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=[checkpoint, printing_cb],
                )

Epoch 1/300
125/125 [==============================] - ETA: 0s - loss: 2.3229 - acc: 0.7823
Epoch 1: loss improved from inf to 2.32294, saving model to model/seq2seq-attention-checkpoint.ckpt
Q: 난 아니라네
A: 

Q: 짝사랑 포기 하려니까 우울해
A: 

Q: 저금 통 깰까
A: 

125/125 [==============================] - 35s 223ms/step - loss: 2.3229 - acc: 0.7823
Epoch 2/300
125/125 [==============================] - ETA: 0s - loss: 1.3129 - acc: 0.8095
Epoch 2: loss improved from 2.32294 to 1.31287, saving model to model/seq2seq-attention-checkpoint.ckpt
125/125 [==============================] - 23s 185ms/step - loss: 1.3129 - acc: 0.8095
Epoch 3/300
125/125 [==============================] - ETA: 0s - loss: 1.2322 - acc: 0.8239
Epoch 3: loss improved from 1.31287 to 1.23216, saving model to model/seq2seq-attention-checkpoint.ckpt
125/125 [==============================] - 23s 186ms/step - loss: 1.2322 - acc: 0.8239
Epoch 4/300
125/125 [==============================] - ETA: 0s - loss: 1.1815 - acc: 0.8261
Epoch 4:

In [ ]:
if not IS_TRAIN_MODE:
    seq2seq.load_weights(CKPT_PATH)

In [ ]:
# 챗봇 실행

while True:
    user_input = input('<< 말을 걸어 보세요!\n')
    if user_input == 'q':
        break
    print(f"<< 사람 질문 : {user_input}")
    print('>> 챗봇 응답: {}'.format(run_chatbot(user_input, seq2seq)))

Reference

- 파이썬 딥러닝 텐서플로 (https://github.com/lovedlim/tensorflow/blob/main/Part%205/5.4_sentence_creation.ipynb)